# 📝 Human 문단 분리 및 데이터셋 생성

**목표**: `generated=0` (Human) 데이터의 `full_text`를 `\n` 기준으로 문단 분리 → LLM으로 AI 버전 생성용 데이터셋 구축

In [18]:
import pandas as pd
from tqdm.auto import tqdm

## 📂 데이터 로드

In [19]:
DATA_PATH = '/Users/youngjinson/Downloads/open/train.csv'

df = pd.read_csv(DATA_PATH)
human_df = df[df['generated'] == 0].copy().reset_index(drop=True)

print(f"전체 데이터: {len(df):,}개")
print(f"Human 데이터: {len(human_df):,}개")

전체 데이터: 97,172개
Human 데이터: 89,177개


## ✂️ 문단 분리 (`\n` 기준)

In [20]:
def split_by_newline(text):
    """줄바꿈(\n) 기준 분리, 빈 줄 제거"""
    return [p.strip() for p in text.split('\n') if p.strip()]

# 전체 Human 데이터 문단 분리
all_paragraphs = []
source_ids = []       # 원본 글 인덱스 (ID)
source_titles = []    # 원본 글 제목
para_indices = []     # 글 내 문단 순서

for doc_id, row in tqdm(human_df.iterrows(), total=len(human_df), desc="문단 분리"):
    paras = split_by_newline(row['full_text'])
    for para_idx, para in enumerate(paras):
        all_paragraphs.append(para)
        source_ids.append(doc_id)
        source_titles.append(row['title'])
        para_indices.append(para_idx)

print(f"\n총 문단 수: {len(all_paragraphs):,}개")
print(f"평균 문단/글: {len(all_paragraphs)/len(human_df):.1f}개")

문단 분리:   0%|          | 0/89177 [00:00<?, ?it/s]


총 문단 수: 1,125,652개
평균 문단/글: 12.6개


## 💾 저장

In [21]:
para_df = pd.DataFrame({
    'para_id': range(len(all_paragraphs)),   # 문단 고유 ID
    'source_doc_id': source_ids,              # 원본 문서 ID
    'para_index': para_indices,               # 문서 내 문단 순서 (0, 1, 2, ...)
    'source_title': source_titles,            # 원본 문서 제목
    'paragraph': all_paragraphs               # 실제 문단 텍스트
})

output_path = '/Users/youngjinson/Downloads/open/human_paragraphs.csv'
para_df.to_csv(output_path, index=False)

print(f"✅ 저장 완료: {output_path}")
print(f"   총 {len(para_df):,}개 Human 문단")

✅ 저장 완료: /Users/youngjinson/Downloads/open/human_paragraphs.csv
   총 1,125,652개 Human 문단


In [22]:
# 샘플 확인
para_df.head(10)

,para_id,source_doc_id,para_index,source_title,paragraph
0,0,0,0,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...
1,1,0,1,카호올라웨섬,마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 ...
2,2,0,2,카호올라웨섬,1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬...
3,3,0,3,카호올라웨섬,1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 ...
4,4,0,4,카호올라웨섬,1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을...
5,5,0,5,카호올라웨섬,1941년 12월 7일에 일어난 일본 제국 해군의 진주만 공격을 계기로 카호올라웨섬...
6,6,1,0,청색거성,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은..."
7,7,1,1,청색거성,"용어는 각자 다른 진화 단계에 있는 여러 가지 별에 적용되는데, 이들 모두 주계열에..."
8,8,1,2,청색거성,"청색거성이라는 명칭은 종종 매우 크고 뜨거운 주계열성과 같이, 다른 무겁고 밝은 별..."
9,9,1,3,청색거성,청색거성은 엄격히 정의된 단어가 아니어서 서로 다른 다양한 유형의 별에 폭넓게 사용...


In [23]:
para_df

,para_id,source_doc_id,para_index,source_title,paragraph
0,0,0,0,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...
1,1,0,1,카호올라웨섬,마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 ...
2,2,0,2,카호올라웨섬,1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬...
3,3,0,3,카호올라웨섬,1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 ...
4,4,0,4,카호올라웨섬,1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을...
...,...,...,...,...,...
1125647,1125647,89176,5,펩시 스터프,펩시 스터프 프로모션 이후에 펩시와 코카-콜라 모두 몇 년 동안에 걸쳐 원래의 캠페...
1125648,1125648,89176,6,펩시 스터프,코카-콜라 컴퍼니는 2005년에 소비자들이 캐나다에서 패키지에 인쇄된 포인트를 수집...
1125649,1125649,89176,7,펩시 스터프,펩시코는 2008년 2월 1일에 아마존 MP3(나중에 아마존 뮤직으로 이름을 바꿈)...
1125650,1125650,89176,8,펩시 스터프,2015년에 펩시 패스로 재출시된 이 프로그램은 다양한 방법으로 소비자가 포인트를 ...
